C:\Users\Hani\AppData\Local\Temp\ipykernel_27112\2229711948.py:76: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((self.width, self.height), Image.ANTIALIAS)


In [2]:
import csv
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


# Create a Scrollable Frame class
class ScrollableFrame(tk.Frame):
    def __init__(self, container, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        canvas = tk.Canvas(self)
        scrollbar = ttk.Scrollbar(self, orient="vertical", command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")


# Read coffee data from a CSV file
def read_coffee_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        coffee_data = list(reader)
        print("Data read from CSV file:", coffee_data)
        return coffee_data


class CoffeeConnectApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Coffee Connect")

        # Create and configure GUI elements
        self.label = tk.Label(self.root, text="Welcome to Coffee Connect!")
        self.label.pack()

        self.button_explore = tk.Button(self.root, text="Explore Coffee Options", command=self.explore_coffee)
        self.button_explore.pack()

        self.button_create_account = tk.Button(self.root, text="Create Account", command=self.create_account)
        self.button_create_account.pack()

        # Load coffee data
        self.coffee_data = read_coffee_data('coffee_data.csv')

    def explore_coffee(self):
        # Creating a new window for displaying coffee options
        window = tk.Toplevel(self.root)
        window.title("Coffee Options")

        scrollable_frame = ScrollableFrame(window)
        scrollable_frame.pack()

        print("Exploring coffee options...")

        # Add coffee options information to the text box
        for data in self.coffee_data:
            print("Current data:", data)
            shop_name = data['shop_name']
            shop_address = data['shop_address']
            name = data['name']
            coffee_type = data['type']
            description = data['description']
            rating = data['rating']
            brand = data['brand']
            ingredient = data['ingredient']
            image_file = data['image']

            # I decided to create a sub frame for each coffee option
            sub_frame = ttk.Frame(scrollable_frame.scrollable_frame)
            sub_frame.pack(pady=10)

            # Display coffee options in a text box
            text_box = tk.Text(sub_frame, height=10, width=50)
            text_box.pack(side="left")

            text_box.insert(tk.END, f"Shop Name: {shop_name}\n")
            text_box.insert(tk.END, f"Shop Address: {shop_address}\n")
            text_box.insert(tk.END, f"Name: {name}\n")
            text_box.insert(tk.END, f"Type: {coffee_type}\n")
            text_box.insert(tk.END, f"Description: {description}\n")
            text_box.insert(tk.END, f"Rating: {rating}\n")
            text_box.insert(tk.END, f"Brand: {brand}\n")
            text_box.insert(tk.END, f"Ingredient: {ingredient}\n")
            text_box.insert(tk.END, "----------------\n")

            text_box.configure(state='disabled')

            # Display the image for each coffee option
            image = Image.open(image_file)
            image = image.resize((200, 200))  # Adjust the size as per your requirement
            photo = ImageTk.PhotoImage(image)
            image_label = tk.Label(sub_frame, image=photo)
            image_label.image = photo  # Keep a reference to the image object to prevent it from being garbage collected
            image_label.pack(side="left")

            print(f"Loading image from: {image_file}")

    def create_account(self):
        # Creating a user account
        print("Creating a new account...")

    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    root = tk.Tk()
    app = CoffeeConnectApp(root)
    app.run()


Data read from CSV file: [{'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Cappuccino', 'type': 'Espresso', 'description': 'A traditional Italian coffee drink', 'rating': '4.5', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Milk', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Espresso.PNG'}, {'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Latte', 'type': 'Espresso', 'description': 'A coffee drink made with espresso and steamed milk', 'rating': '4.2', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Steamed Milk', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Latte.PNG'}, {'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Mocha', 'type': 'Espresso', 'description': 'A chocolate-flavored coffee drink', 'rating': '4.0', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Chocolate', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Mocha.PNG'}, {'shop_name': 'Joes Coffee', 'shop_address': '456 

In [1]:
import csv
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


# Create a Scrollable Frame class
class ScrollableFrame(tk.Frame):
    def __init__(self, container, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        canvas = tk.Canvas(self)
        scrollbar = ttk.Scrollbar(self, orient="vertical", command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")


# Read coffee data from a CSV file
def read_coffee_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        coffee_data = list(reader)
        print("Data read from CSV file:", coffee_data)
        return coffee_data


class CoffeeConnectApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Coffee Connect")

        # Create and configure GUI elements
        self.label = tk.Label(self.root, text="Welcome to Coffee Connect!")
        self.label.pack()

        self.button_explore = tk.Button(self.root, text="Explore Coffee Options", command=self.explore_coffee)
        self.button_explore.pack()

        self.button_create_account = tk.Button(self.root, text="Create Account", command=self.create_account)
        self.button_create_account.pack()

        # Load coffee data
        self.coffee_data = read_coffee_data('coffee_data.csv')

        # Store references to PhotoImage objects
        self.photo_images = []

    def explore_coffee(self):
        # Creating a new window for displaying coffee options
        window = tk.Toplevel(self.root)
        window.title("Coffee Options")

        scrollable_frame = ScrollableFrame(window)
        scrollable_frame.pack()

        print("Exploring coffee options...")

        # Add coffee options information to the text box
        for data in self.coffee_data:
            print("Current data:", data)
            shop_name = data['shop_name']
            shop_address = data['shop_address']
            name = data['name']
            coffee_type = data['type']
            description = data['description']
            rating = data['rating']
            brand = data['brand']
            ingredient = data['ingredient']
            image_file = data['image']

            # I decided to create a sub frame for each coffee option
            sub_frame = ttk.Frame(scrollable_frame.scrollable_frame)
            sub_frame.pack(pady=10)

            # Display coffee options in a text box
            text_box = tk.Text(sub_frame, height=10, width=50)
            text_box.pack(side="left")

            text_box.insert(tk.END, f"Shop Name: {shop_name}\n")
            text_box.insert(tk.END, f"Shop Address: {shop_address}\n")
            text_box.insert(tk.END, f"Name: {name}\n")
            text_box.insert(tk.END, f"Type: {coffee_type}\n")
            text_box.insert(tk.END, f"Description: {description}\n")
            text_box.insert(tk.END, f"Rating: {rating}\n")
            text_box.insert(tk.END, f"Brand: {brand}\n")
            text_box.insert(tk.END, f"Ingredient: {ingredient}\n")
            text_box.insert(tk.END, "----------------\n")

            text_box.configure(state='disabled')

            # Display the image for each coffee option
            image = Image.open(image_file)
            image = image.resize((200, 200))  # Adjust the size as per your requirement
            photo = ImageTk.PhotoImage(image)
            self.photo_images.append(photo)  # Store the PhotoImage object
            image_label = tk.Label(sub_frame, image=photo)
            image_label.image = photo  # Keep a reference to the image object to prevent it from being garbage collected
            image_label.pack(side="left")

            print(f"Loading image from: {image_file}")

    def create_account(self):
        # Creating a user account
        print("Creating a new account...")

    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    root = tk.Tk()
    app = CoffeeConnectApp(root)
    app.run()


Data read from CSV file: [{'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Cappuccino', 'type': 'Espresso', 'description': 'A traditional Italian coffee drink', 'rating': '4.5', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Milk', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Espresso.PNG'}, {'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Latte', 'type': 'Espresso', 'description': 'A coffee drink made with espresso and steamed milk', 'rating': '4.2', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Steamed Milk', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Latte.PNG'}, {'shop_name': 'Starbucks', 'shop_address': '123 Main St', 'name': 'Mocha', 'type': 'Espresso', 'description': 'A chocolate-flavored coffee drink', 'rating': '4.0', 'brand': 'Starbucks Brand', 'ingredient': 'Espresso and Chocolate', 'image': 'd:\\Project584\\CoffeeConnect_HCI584\\image\\Mocha.PNG'}, {'shop_name': 'Joes Coffee', 'shop_address': '456 

In [3]:
import tkinter as tk
from tkinter import Tk, Canvas, Frame, Scrollbar, Button, Label, VERTICAL, BOTH, NSEW, NW
from PIL import Image, ImageTk
import csv

class CoffeeConnect:
    def __init__(self, description_dict, width=500, height=500):
        self.root = Tk()

        self.load_images_button = Button(self.root, text="Load Images", command=self.Coffee_Options)
        self.load_images_button.grid(row=0, column=0, columnspan=2)  

        self.create_account_button = Button(self.root, text="Create Account", command=self.create_account)
        self.create_account_button.grid(row=1, column=0, columnspan=2)  

        self.canvas = Canvas(self.root)
        self.canvas.grid(row=2, column=0, sticky=NSEW) 

        self.scrollbar = Scrollbar(self.root, orient=VERTICAL, command=self.canvas.yview)
        self.scrollbar.grid(row=2, column=1, sticky='ns')  

        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.frame = Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.frame, anchor=NW)

        self.img_paths = description_dict['image']
        self.descriptions = [dict(zip(description_dict, col)) for col in zip(*description_dict.values())]
    
        self.width = width
        self.height = height

        self.root.grid_rowconfigure(2, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

    def create_account(self):
        print("Creating a new account...")

    def run(self):
        self.root.mainloop()

    def Coffee_Options(self):
        for widget in self.frame.winfo_children():
            widget.destroy()

        for img_path, description_dict in zip(self.img_paths, self.descriptions):
            img = Image.open(img_path)
            img = img.resize((self.width, self.height), Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(img)
            label = Label(self.frame, image=photo)
            label.image = photo
            label.pack(anchor='center', pady=10)

            desc_text = '\n'.join([f'{k}: {v}' for k, v in description_dict.items()])
            desc_label = Label(self.frame, text=desc_text, justify='center')
            desc_label.pack(anchor='center', pady=10)

        self.frame.update_idletasks()
        self.canvas.configure(scrollregion=self.canvas.bbox('all'))     

def read_csv_to_dictionary(csv_file_path):
    data_dict = {}
    
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        
        for row in reader:
            for column, value in row.items():
                if column not in data_dict:
                    data_dict[column] = []
                
                data_dict[column].append(value)
    
    return data_dict

data_dict = read_csv_to_dictionary('coffee_data.csv')
app = CoffeeConnect(data_dict)
app.run()


C:\Users\Hani\AppData\Local\Temp\ipykernel_14636\3147575696.py:48: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((self.width, self.height), Image.ANTIALIAS)


Creating a new account...


In [5]:
import tkinter as tk
from tkinter import Tk, Canvas, Frame, Scrollbar, Button, Label, VERTICAL, BOTH, NSEW, NW
from PIL import Image, ImageTk
import csv

class CoffeeConnect:
    def __init__(self, description_dict, width=500, height=500):
        self.root = Tk()

        self.load_images_button = Button(self.root, text="Load Images", command=self.Coffee_Options)
        self.load_images_button.grid(row=0, column=0, columnspan=2)  

        self.create_account_button = Button(self.root, text="Create Account", command=self.create_account)
        self.create_account_button.grid(row=1, column=0, columnspan=2)  

        self.canvas = Canvas(self.root)
        self.canvas.grid(row=2, column=0, sticky=NSEW) 

        self.scrollbar = Scrollbar(self.root, orient=VERTICAL, command=self.canvas.yview)
        self.scrollbar.grid(row=2, column=1, sticky='ns')  

        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.frame = Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.frame, anchor=NW)

        self.img_paths = description_dict['image']
        self.descriptions = [dict(zip(description_dict, col)) for col in zip(*description_dict.values())]
    
        self.width = width
        self.height = height

        self.root.grid_rowconfigure(2, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

    def create_account(self):
        self.new_account_window = tk.Toplevel(self.root)
        self.new_account_window.title("Create account")

        tk.Label(self.new_account_window, text="Email").grid(row=0, column=0)
        self.email_entry = tk.Entry(self.new_account_window)
        self.email_entry.grid(row=0, column=1)

        tk.Label(self.new_account_window, text="Password").grid(row=1, column=0)
        self.password_entry = tk.Entry(self.new_account_window, show="*")
        self.password_entry.grid(row=1, column=1)

        submit_button = tk.Button(self.new_account_window, text="Submit", command=self.submit_new_account)
        submit_button.grid(row=2, column=0, columnspan=2)

    def submit_new_account(self):
        entered_email = self.email_entry.get()
        entered_password = self.password_entry.get()
        print(f"Entered email: {entered_email}, entered password: {entered_password}")
        self.new_account_window.destroy()

    def run(self):
        self.root.mainloop()

    def Coffee_Options(self):
        for widget in self.frame.winfo_children():
            widget.destroy()

        for img_path, description_dict in zip(self.img_paths, self.descriptions):
            img = Image.open(img_path)
            img = img.resize((self.width, self.height), Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(img)
            label = Label(self.frame, image=photo)
            label.image = photo
            label.pack(anchor='center', pady=10)

            desc_text = '\n'.join([f'{k}: {v}' for k, v in description_dict.items()])
            desc_label = Label(self.frame, text=desc_text, justify='center')
            desc_label.pack(anchor='center', pady=10)

        self.frame.update_idletasks()
        self.canvas.configure(scrollregion=self.canvas.bbox('all'))     

def read_csv_to_dictionary(csv_file_path):
    data_dict = {}
    
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        
        for row in reader:
            for column, value in row.items():
                if column not in data_dict:
                    data_dict[column] = []
                
                data_dict[column].append(value)
    
    return data_dict

data_dict = read_csv_to_dictionary('coffee_data.csv')
app = CoffeeConnect(data_dict)
app.run()


Entered email: , entered password: 


C:\Users\Hani\AppData\Local\Temp\ipykernel_14636\307774203.py:66: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((self.width, self.height), Image.ANTIALIAS)
